# importing necesary libraries

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.collections as mcoll
import matplotlib.cm as cm


# Abstract methods
here i create abstract methods for solver, and functions

In [ ]:
class Solver(ABC):
    """A solver. It may be initialized with some hyperparameters."""
    @abstractmethod
    def get_parameters(self):
        """Returns a dictionary of hyperparameters"""
        pass
    @abstractmethod
    def solve(self, problem, x0, *args, **kwargs):
        """
        A method that solves the given problem for given initial solution.
        It may accept or require additional parameters.
        Returns the solution and may return additional info.
        """
        pass

class Function(ABC):
    """A function. It may be initialized with some parameters."""
    @abstractmethod
    def __call__(self, x, *args, **kwargs):
        """
        A method that returns the value of the function at given point.
        It may accept or require additional parameters.
        """
        pass
    @abstractmethod
    def gradient(self, x, *args, **kwargs):
        """
        A method that returns the gradient of the function at given point.
        It may accept or require additional parameters.
        """
        pass

the purpouse of class AnyFunction is to just hold both the normal function and its gradient in one callable obcect

In [ ]:
class AnyFunction(Function):
    def __init__(self, equation, gradient):
        self.equation = equation
        self.gradient_func = gradient
    def __call__(self, x, *args, **kwargs):
        return self.equation(x)
    def gradient(self, x, *args, **kwargs):
        return self.gradient_func(x)


# Gradient Decent solver
in the code below i implemtented a simple gradient decent solver that for every point that for a number of iterations it is currently in calculates the gradient of the function and then substracts the gradient*stepsize from the current position

- I also experimented with early_stop: when it is active the solver will stop looking for a minimum if the length af the gradient vector is shorter then early_stop value, it is good if the function changes somewhat rapidly in everywhere, rather then in minimum, but very bad when the function has areas where it is still, doesn't change too much, because when the solver gets started in such a point then the solver will immidietly stop solving for minimums

In [ ]:
class GradientDescentSolver(Solver):
    def __init__(self, step_size=0.1, max_iter=1000, early_stop=None):
        self.step_size = step_size
        self.max_iter = max_iter
        self.stop = early_stop

    def set_step_size(self, new_step_size):
        self.step_size = new_step_size

    def set_early_stop(self, new_early_stop):
        self.stop = new_early_stop

    def set_max_iter(self, new_max_iter):
        self.max_iter = new_max_iter

    def get_parameters(self):
        return {"step_size": self.step_size, "max_iter": self.max_iter, "early_stop": self.stop, "get_angry": self.get_angry}

    def solve(self, function, x0):
        x = x0.copy()

        history = []
        history.append(x.copy())

        for i in range(self.max_iter):
            grad = function.gradient(x)

            if self.stop is not None and np.linalg.norm(grad) < self.stop:
                break

            step = self.step_size * grad

            step_lenght = (np.linalg.norm(step))

            x -= step
            history.append(x.copy())
        return x, history

# Difine functions
- here i define functions that were given in excersize file, make an object for each of them to be able to hold both the normal functions and their gradients in one object
- then i initialize solver object with some arguments, for the first function (1D) it is best to not youse get_angry option as it increases the step_size to infinity, i chose to not hard code any limitations on how big it can get, to not make the solution too complex (also it is just a mode i experimented with, not the original task)
- this solver can solve both functions without changing anything in the solver

In [ ]:
def function1(x):
    return 2 * x**2 + 3 * x - 1
def gradient1(x):
    return 4 * x + 3
f = AnyFunction(function1, gradient1)

def function2(x):
    return (1 - 0.6 * np.exp(-x[0]**2 - x[1]**2) - 0.4 * np.exp(-(x[0] + 1.75)**2 - (x[1] - 1)**2))
def gradient2(x):
    return np.array(
        [
            1.2 * x[0] * np.exp(-x[0]**2 - x[1]**2) + 0.8 * (x[0] + 1.75) * np.exp(-(x[0] + 1.75)**2 - (x[1] - 1)**2),
            1.2 * x[1] * np.exp(-x[0]**2 - x[1]**2) + 0.8 * (x[1] - 1) * np.exp(-(x[0] + 1.75)**2 - (x[1] - 1)**2)
        ])
g = AnyFunction(function2, gradient2)

# Functions visualization
- here i plot using matplotlib functions that the solver will later solve (find their minimums)

In [ ]:
range_2D = 3
range_1D = 20


x = np.linspace(-range_1D, range_1D, 100)
y = f(x)
fig, ax = plt.subplots()
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
norm = plt.Normalize(y.min(), y.max())
lc = mcoll.LineCollection(segments, cmap='coolwarm', norm=norm)
lc.set_array(y)
lc.set_linewidth(2)
line = ax.add_collection(lc)
ax.set_title('Plot of f(x)')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid(True, alpha=0.5)
fig.colorbar(line, ax=ax, label='f(x)')
ax.set_facecolor('#f0f0f0')
plt.xlim(x.min(), x.max())
plt.ylim(y.min(), y.max())
plt.show()


X, Y = np.meshgrid(np.linspace(-range_2D, range_2D, 100), np.linspace(-range_2D, range_2D, 100))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, g([X, Y]), cmap='coolwarm')
plt.title('Plot of g(x, y)')
fig.colorbar(surf, ax=ax, label='g(x, y)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

# generate a, b random starting points for each function

In [ ]:
a = 10 # number of points for 1D function
b = 10 # number of points for 2D function

np.random.seed(100) # for reproducibility, comment out for random results

random_points_f = np.random.uniform(-range_1D, range_1D, size=(a, 1))
random_points_g = np.random.uniform(-range_2D, range_2D, size=(b, 2))

## Display the generated points

In [ ]:
colors = f(random_points_f)
plt.scatter(random_points_f, f(random_points_f), c=colors, s=40, cmap='viridis', edgecolors='black')
plt.gca().axes.get_yaxis().set_visible(False)
plt.title('Random Points for f(x)')
plt.xlabel('X')
plt.colorbar(label='f(x)')
plt.grid(True, alpha=0.5)
plt.show()

colors = g(random_points_g.T)
plt.scatter(random_points_g[:, 0], random_points_g[:, 1], c=colors, s=40, cmap='viridis', edgecolors='black')
plt.title('Random Points for g(x, y)')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar(label='g(x, y)')
plt.grid(True, alpha=0.5)
plt.show()

# Define functions that will solve the functions for minimums for each step size and each start point

In [ ]:
def plot_and_save_2D(function, no_step_sizes, range_of_step_sizes, random_points, solver, filename):
    n = no_step_sizes
    m = len(random_points)

    step_sizes = np.random.uniform(range_of_step_sizes[0], range_of_step_sizes[1], n)
    step_sizes = np.sort(step_sizes)

    x = np.linspace(-3, 3, 100)
    y = np.linspace(-3, 3, 100)
    X, Y = np.meshgrid(x, y)
    Z = function([X, Y])

    fig, axs = plt.subplots(n, m, figsize=(10*m, 10*n), subplot_kw={'projection': '3d'})
    for i, step_size in enumerate(step_sizes):
        solver.set_step_size(step_size)
        for j, point in enumerate(random_points):
            _, data = solver.solve(function, point)

            points = np.array(data)
            points_Z = function([points[:, 0], points[:, 1]])

            ax = axs[i, j]
            ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.5)
            ax.plot(points[:, 0], points[:, 1], points_Z, color='blue')

            ax.scatter(points[:, 0], points[:, 1], points_Z, color='red', s=10, alpha=0.5)
            ax.scatter(points[0, 0], points[0, 1], points_Z[0], color='green', s=100)
            ax.scatter(points[-1, 0], points[-1, 1], points_Z[-1], color='black', s=100)
            ax.set_title(f'Step size: {step_size:.3f}, Starting point: {point}, number of iterations: {len(data)}')
    fig.savefig(filename)
    plt.close(fig)

In [ ]:
def plot_and_save_1D(function, no_step_sizes, range_of_step_sizes, random_points, solver, filename):
    n = no_step_sizes
    m = len(random_points)

    step_sizes = np.random.uniform(range_of_step_sizes[0], range_of_step_sizes[1], n)
    step_sizes = np.sort(step_sizes)


    fig, axs = plt.subplots(n, m, figsize=(10*m, 10*n))
    for i, step_size in enumerate(step_sizes):
        solver.set_step_size(step_size)
        for j, point in enumerate(random_points):
            _, data = solver.solve(function, point)

            points = np.array(data)
            points_y = function(points)

            x_max = max(points)

            if x_max < range_1D:
                x_max = range_1D

            x = np.linspace(-x_max, x_max, 100)
            y = function(x)

            ax = axs[i, j]
            ax.plot(x, y, color='blue', alpha=0.5, linewidth=5)
            ax.plot(points, points_y, color='red', linestyle='--')

            ax.scatter(points, points_y, color='red', s=10, alpha=0.5)
            ax.scatter(points[0], points_y[0], color='green', s=100)
            ax.scatter(points[-1], points_y[-1], color='black', s=100)
            ax.set_title(f'Step size: {step_size:.3f}, Starting point: {point}, number of iterations: {len(data)}')
    fig.savefig(filename)
    plt.close(fig)

# Generate the plot files
- I made a dicision to not display the results in the notebook as the generated file can be quite big, so it is saved in a currents working category with the name specified by filename
- the plots can be generated using code below (it is best to not increase the no_step_sizes or number of random points as it will take a long time to generate and the file will be very big in size so depends on the computer it can be even impossible to display)

## for 1D function
- it can be noticed that after step_size value exceeds about 0.5 algorythm doesn't find a minimum, but just explodes in value
- also I noticed that for such a simple function a step_size of about 0.3 solves the funciton is the least ammount of iterations when early_stop is active
- but the most reliable from the plots seemes to be 0,1 a good deal beetwen small no_of_itrerations and not too big steps


In [ ]:
solver = GradientDescentSolver(step_size=0.1, max_iter=100, early_stop=1e-4)
plot_and_save_1D(function=f, no_step_sizes=20, range_of_step_sizes=[0.001, 1], random_points=random_points_f, solver=solver, filename='gradient_descent_1D.png')

## for 2D function

- with early stop the functiton sometimes saves some iterations that wouldn't imrpove minimum by a big margin, but if the point is in a still region of the function it can terminate solve procces right away, making it impossible for the algorythm to ever find any minimum (it can be mitigated by setting early stop to about 1e-6 but then it often doesn't stop even after algoryth has reached minimum)
- for this reason i also generated a file wihout early_stopping, but in it i only tested the range of [0, 1] step_size

In [ ]:
# with early stop
solver.set_max_iter(1000)

plot_and_save_2D(function=g, no_step_sizes=20, range_of_step_sizes=[0.01, 2], random_points=random_points_g, solver=solver, filename='gradient_descent_2D_with_early_stop.png')

In [ ]:
# without early stop
solver.set_early_stop(None)

plot_and_save_2D(function=g, no_step_sizes=10, range_of_step_sizes=[0.01, 1], random_points=random_points_g, solver=solver, filename='gradient_descent_2D.png')

- it can be noticed that bigger step sizes with early_stopping can make the algorythm find the minimum faster, but too big (more then 1,5) can make it never find a minimum becouse of too large step size
- i would say that for the 2D funciton a step size of about 0,5 seems the best at solving the function reliably and for most points